# 곧바로 난독화 텍스트와 복원된 텍스트셋 학습시키기

### 인코더 사전 훈련후, 인코더 -> 디코더 파인튜닝

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sbs

In [2]:
from tokenizers import Tokenizer

# 저장된 토크나이저 로드
tokenizer = Tokenizer.from_file("tokenizers/BPE_tokenizer_50000.json")
text_sample =  '별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.'
# 테스트
input_encoded = tokenizer.encode(text_sample)
print("토큰화 결과:", input_encoded.tokens)


토큰화 결과: ['별', '한', '게토', '았깝', '땀', '.', '왜', '싸람', '듯', '릭', '펼', '1캐', '를', '쥰눈', '징', '컥', '꺾', '폰', '싸람', '믐', '롯', '섞', '맒록', '섧멍', '핥', '쟈', '닐', '탯', '끎룐눈', '녀뮤', '퀼', '교', '...', '야뭍', '툰', '둠', '변', '닺씨', '깍', '낄', '싫훈', '굣', '.', '깸삥', '읊', '20', '여', '년', '댜녁', '뵨', '곧', '중', '쩨윌', '귑푼', '낙', '팠', '떤', '곶', '.']


In [3]:
tokenizer.get_vocab_size()

50000

In [4]:
from transformers import BartTokenizerFast

tokenizer2 = BartTokenizerFast.from_pretrained("facebook/bart-base")
tokenizer2

/home/jeongyuseong/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}
)

In [5]:
import random

def apply_infilling_masking(text, mask_token="<mask>", mask_prob=0.15, max_mask_size=3):
    """
    Infilling Masking을 적용하는 함수.
    
    Args:
        text (str): 원본 텍스트
        mask_token (str): 마스킹 토큰 (디폴트: <mask>)
        mask_prob (float): 토큰을 마스킹할 확률 (디폴트: 0.15)
        max_mask_size (int): 최대 연속 마스킹 토큰 수 (디폴트: 3)

    Returns:
        str: 마스킹된 텍스트
    """
    # 텍스트를 공백 기준으로 토큰화
    tokens = text.split()

    # 마스킹 대상 토큰 선택
    num_masks = max(1, int(len(tokens) * mask_prob))
    mask_positions = random.sample(range(len(tokens)), num_masks)

    # 마스킹 적용
    for pos in mask_positions:
        mask_length = random.randint(1, max_mask_size)  # 연속 마스크 길이
        tokens[pos:pos + mask_length] = [mask_token]

    # 마스킹된 텍스트 반환
    return " ".join(tokens)


# 테스트
original_texts = [
    "꺼율위 쓱팍",
    "탐패 냄쌕갊 묘둔 쟝졈울",
    "집윈 축쳐눌료 딴너왓눈뎁",
    "탐패 냄쌕갊 묘둔 쟝졈울 카셩뷔",
]
masked_text_list = []

# Infilling Masking 적용 예시
for text in original_texts:
    masked_text = apply_infilling_masking(text)
    masked_text_list.append(masked_text)
    print(f"원본 텍스트: {text}")
    print(f"마스킹된 텍스트: {masked_text}")
    print("-" * 50)

원본 텍스트: 꺼율위 쓱팍
마스킹된 텍스트: <mask>
--------------------------------------------------
원본 텍스트: 탐패 냄쌕갊 묘둔 쟝졈울
마스킹된 텍스트: <mask> 묘둔 쟝졈울
--------------------------------------------------
원본 텍스트: 집윈 축쳐눌료 딴너왓눈뎁
마스킹된 텍스트: 집윈 <mask>
--------------------------------------------------
원본 텍스트: 탐패 냄쌕갊 묘둔 쟝졈울 카셩뷔
마스킹된 텍스트: 탐패 냄쌕갊 묘둔 쟝졈울 <mask>
--------------------------------------------------


In [6]:
for text in masked_text_list:
    tokenized = tokenizer.encode(text)
    print(f"원본 텍스트: {text}")
    print(f"토큰화된 텍스트: {tokenized.tokens}")
    print("-" * 50)
print()
for text in original_texts:
    tokenized = tokenizer.encode(text)
    print(f"원본 텍스트: {text}")
    print(f"토큰화된 텍스트: {tokenized.tokens}")
    print("-" * 50)


원본 텍스트: <mask>
토큰화된 텍스트: ['<mask>']
--------------------------------------------------
원본 텍스트: <mask> 묘둔 쟝졈울
토큰화된 텍스트: ['<mask>', '묘둔', '쟝졈', '울']
--------------------------------------------------
원본 텍스트: 집윈 <mask>
토큰화된 텍스트: ['집', '윈', '<mask>']
--------------------------------------------------
원본 텍스트: 탐패 냄쌕갊 묘둔 쟝졈울 <mask>
토큰화된 텍스트: ['탐패', '냄쌕', '갊', '묘둔', '쟝졈', '울', '<mask>']
--------------------------------------------------

원본 텍스트: 꺼율위 쓱팍
토큰화된 텍스트: ['꺼', '율위', '쓱', '팍']
--------------------------------------------------
원본 텍스트: 탐패 냄쌕갊 묘둔 쟝졈울
토큰화된 텍스트: ['탐패', '냄쌕', '갊', '묘둔', '쟝졈', '울']
--------------------------------------------------
원본 텍스트: 집윈 축쳐눌료 딴너왓눈뎁
토큰화된 텍스트: ['집', '윈', '축', '쳐', '눌료', '딴너왓', '눈뎁']
--------------------------------------------------
원본 텍스트: 탐패 냄쌕갊 묘둔 쟝졈울 카셩뷔
토큰화된 텍스트: ['탐패', '냄쌕', '갊', '묘둔', '쟝졈', '울', '카셩뷔']
--------------------------------------------------


In [7]:
test_path = 'datas/test.csv'
train_path = 'datas/train.csv'

test_pd = pd.read_csv(test_path)
train_pd = pd.read_csv(train_path)

# 가장 긴 생성할 테스트 난독화 텍스트 길이 : 1965
# 두 문장 모두 최대길이는 1381 1381
train_text = list(train_pd['input'])
masked_text = [apply_infilling_masking(x) for x in train_text]
textGT = list(train_pd['output'])


In [8]:
for i in range(3):
    print(train_text[i])
    print(masked_text[i])
    print(textGT[i])
    print("-" * 50)

별 한 게토 았깝땀. 왜 싸람듯릭 펼 1캐를 쥰눈징 컥꺾폰 싸람믐롯섞 맒록 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 변 닺씨 깍낄 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.
별 <mask> 게토 았깝땀. <mask> 펼 1캐를 쥰눈징 컥꺾폰 <mask> 섧멍핥쟈닐 탯끎룐눈 녀뮤 퀼교... 야뭍툰 둠 <mask> 싫훈 굣. 깸삥읊 20여 년 댜녁뵨 곧 중 쩨윌 귑푼 낙팠떤 곶.
별 한 개도 아깝다. 왜 사람들이 별 1개를 주는지 겪어본 사람으로서 말로 설명하자니 댓글로는 너무 길고... 아무튼 두 번 다시 가길 싫은 곳. 캠핑을 20여 년 다녀본 곳 중 제일 기분 나빴던 곳.
--------------------------------------------------
잚많 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ
<mask> 쟉꼬 갉 태 좋눼욥. 차못동 줆 ㅋ
잠만 자고 갈 때 좋네요. 잠옷도 줌 ㅋ
--------------------------------------------------
절테 간면 않 된는 굣 멥몫
절테 간면 <mask> 된는 굣 멥몫
절대 가면 안 되는 곳 메모
--------------------------------------------------


In [9]:
input_encoded = tokenizer.encode(original_texts[0])
print(input_encoded.tokens)
print(input_encoded.attention_mask)
print(input_encoded.ids)

['꺼', '율위', '쓱', '팍']
[1, 1, 1, 1]
[386, 17635, 1473, 2229]


In [10]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import BartConfig, BartModel, PreTrainedTokenizerFast
from torch.optim import AdamW


class EncoderDataset(Dataset):
    """
    inputs ('str' list): Text infilling된 난독화 텍스트 리스트
    targets ('str' list): 원본 난독화 텍스트 리스트
    tokenizer : 커스텀 토크나이저 (BPE, WordPiece 등)
    max_len : 원본 문자열 최대 길이
    """
    def __init__(self, inputs, targets, tokenizer, max_len):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        input_text = self.inputs[index]
        target_text = self.targets[index]

        # 입력 텍스트 토큰화 (타겟 텍스트는 별도로 사용 안 함)
        input_encoded = self.tokenizer.encode(input_text)
        target_encoded = self.tokenizer.encode(target_text)

        # 토큰 ID와 패딩 적용
        input_ids = input_encoded.ids
        target_ids = target_encoded.ids
        attention_mask = [1] * len(input_ids)

        # 시퀀스 길이 조정
        if len(input_ids) < self.max_len:
            # 패딩 추가
            pad_length = self.max_len - len(input_ids)
            target_pad_length = self.max_len - len(target_ids)
            input_ids += [self.tokenizer.token_to_id("<pad>")] * pad_length
            target_ids += [self.tokenizer.token_to_id("<pad>")] * target_pad_length
            attention_mask += [0] * pad_length
        else:
            # 길이 초과 시 자르기
            input_ids = input_ids[:self.max_len]
            target_ids = target_ids[:self.max_len]
            attention_mask = attention_mask[:self.max_len]

        # 텐서로 변환
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        target_ids = torch.tensor(target_ids, dtype=torch.long)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long)

        return {
            "input_ids": input_ids.squeeze(0),
            "attention_mask": attention_mask.squeeze(0),
            "labels": target_ids.squeeze(0)
        }

In [11]:
dataset = EncoderDataset(masked_text, train_text, tokenizer, 2000)
sample = dataset.__getitem__(0)
print(sample['attention_mask'].shape)
print(sample['input_ids'].shape)
print(sample['labels'].shape)

torch.Size([2000])
torch.Size([2000])
torch.Size([2000])


In [12]:
from torch.utils.data import DataLoader, Dataset
from transformers import BartConfig, BartModel, PreTrainedTokenizerFast
from torch.optim import AdamW

dataset = EncoderDataset(masked_text, train_text, tokenizer=tokenizer, max_len=2000)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

batchiter = iter(dataloader)
batch = next(batchiter)
print(batch['input_ids'])
print(batch['attention_mask'])
print(batch['labels'])

tensor([[39716,  7798,   882,  ...,     0,     0,     0],
        [ 2969,  3448,  2506,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[39716,  7798,   882,  ...,     0,     0,     0],
        [ 2969,  3448,  2506,  ...,     0,     0,     0]])


In [13]:
from transformers import BartModel
# 1031
config = BartConfig(
    vocab_size=tokenizer.get_vocab_size(),
    d_model=768,
    encoder_layers=6,
    encoder_attention_heads=8,
    max_position_embeddings=2000
)
model = BartModel(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [14]:
batchiter = iter(dataloader)
batch = next(batchiter)

model.to(device)
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['labels'].to(device)
print(input_ids.shape)
print(attention_mask.shape)
print(labels.shape)

torch.Size([2, 2000])
torch.Size([2, 2000])
torch.Size([2, 2000])


In [15]:
print("input_ids:", batch['input_ids'])
print("최대 토큰 ID:", batch['input_ids'].max().item())
print("모델 vocab_size:", config.vocab_size)

input_ids: tensor([[2077, 3622, 2719,  ...,    0,    0,    0],
        [3512,    4, 2870,  ...,    0,    0,    0]])
최대 토큰 ID: 44463
모델 vocab_size: 50000


In [16]:
import torch.nn as nn


# 선형 레이어 추가
vocab_size = tokenizer.get_vocab_size()  # 예시 vocab_size
linear_layer = nn.Linear(768, vocab_size).to(device)

# 모델 출력 후 변환

outputs = model.encoder(input_ids=input_ids, attention_mask=attention_mask)
logits = outputs.last_hidden_state  # torch.Size([2, 2000, 768])
print(logits.shape)

# 선형 변환을 통해 logits 크기 조정
logits = linear_layer(logits)  # torch.Size([2, 2000, vocab_size])
print(logits.shape)

# 손실 계산을 위한 형태 변환
logits = logits.view(-1, vocab_size)  # torch.Size([4000, vocab_size])
print(logits.shape)
labels = labels.view(-1)              # torch.Size([4000])
print(labels.shape)


torch.Size([2, 2000, 768])
torch.Size([2, 2000, 50000])
torch.Size([4000, 50000])
torch.Size([4000])


In [17]:
# # 예시 텐서: (batch_size * max_length, vocab_size)
# logits = torch.randn(4000, 50000)

# # 1. 가장 높은 확률의 토큰 인덱스 추출
# predicted_token_ids = torch.argmax(logits, dim=-1)  # torch.Size([4000])

# # 2. 토크나이저로 텍스트 복원
# # 예시 토크나이저: 커스텀 토크나이저 사용 가능
# tokenizer = PreTrainedTokenizerFast.from_pretrained('gpt2')

# # batch 크기와 시퀀스 길이 재구성
# batch_size = 2
# max_length = 2000
# predicted_token_ids = predicted_token_ids.view(batch_size, max_length)

# # 3. 토큰 인덱스를 텍스트로 변환
# decoded_texts = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

# # 출력 확인
# for i, text in enumerate(decoded_texts):
#     print(f"Decoded Text {i + 1}:")
#     print(text)
#     print("-" * 50)


### to do list

원본 텍스트 -> 내가정의한데이터셋 -> 바트모델 -> 선형모델 -> 손실함수 계산

#### 모델 재정의
바트모델 -> 선형모델 구조로 재정의하기.

In [ ]:
# 인코더 재정의
import torch.nn as nn
from transformers import BartModel, BartConfig

class CustomBartModel(nn.Module):
    def __init__(self, config, vocab_size):
        super(CustomBartModel, self).__init__()
        self.bart = BartModel(config)
        self.output_layer = nn.Linear(config.d_model, vocab_size)  # 최종 출력 레이어 추가

    def forward(self, input_ids, attention_mask):
        outputs = self.bart.encoder(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.output_layer(outputs.last_hidden_state)  # torch.Size([batch_size, seq_len, vocab_size])
        return logits


# 모델 초기화
config = BartConfig(vocab_size=tokenizer.get_vocab_size(), d_model=768, encoder_layers=6, encoder_attention_heads=8, max_position_embeddings=2000)
model = CustomBartModel(config, vocab_size=tokenizer.get_vocab_size()).to(device)

# 옵티마이저 정의
optimizer = AdamW(model.parameters(), lr=5e-5)

# 훈련 루프 예시
for epoch in range(epochs):
    model.train()
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 순전파
        logits = model(input_ids, attention_mask)

        # 손실 계산
        logits = logits.view(-1, logits.size(-1))
        labels = labels.view(-1)
        loss = criterion(logits, labels)

        # 역전파 및 가중치 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


## 결과확인

In [18]:
sample_logic = logits.clone().detach()
predicted_token_ids = torch.argmax(sample_logic, dim=-1)  # torch.Size([4000])
for token_id in predicted_token_ids:
    text = tokenizer.id_to_token(token_id)
    print(text,end=' ')

합닉따 옥촛 천홧 구런쥐 갔야오 캔 깐능 써피 퍼뉘 엊딘 합뉜따 뉴꾼 꿇눈 신껑 뜩썽 었씁뉘닺 악췸 샬먼 돋때쩨 퓌 걔슈 룔롬 뎄써 깃떼 측 단굘 일랴먼 탬뮨 숲뉘닷 갔숩님따 쑴 앉쿄 쓱갑 빗헤 쀼랖 욕씰 할룸 뻔운 까윕 떡 702효 빻로 냠몇 쥬뻔 묠레 앉항 702효 빻로 총루 했쑴 햐먼셔 랐눈 쥐냇 쩜문 베닯 실는 둘역 까윕 앉닒먼 뒈엶 베닯 욕씰 환뷸헤 딸랸 을로 엘설 쫍았 답뵤 빻로 실는 까윕 라꼬오 셔피 갹꺄어셔 까윕 갇따용 있엾 왓따 쿱뷔 또욕 맑숨 적긁 웍 좋앗움 횟 욕졸 맞셰오 페셔 위뿐 묽일 좋얄오 앍 끓로 겄토 한따먼 쭌댜 페졍 걋탸 좋앗움 쌈퓨 쵸흖 욕씰 꼿익 o 떼익 캅닐댜 1쉬깐 땅황 횟 꼿많 빻로 햐긷 있쑴 맛 잎풂 견쪼 빵무 좋공 총루 탄췌 쉽닒댜 라꼬오 혼 찐넬 웩께 갔두 륏캬략 땟중 쥑껀 만힙 쫏식됴 환뷸헤 욕씰 라꼬오 헤셥 좋았씁니따 터렵교 쒼설 횟 빵무 십였 휵긺를 뤼얻 슛 엷씸휘 계올 만힙 역깁 귀본 좋았씁니따 긋렇 옥촛 빠닭 총루 헷아 될교 씩크 6 라꼬오 넌뮤 밟켠 겄토 음룝 멓뮬 맞윗눈 겄토 베닯 빵무 껄뤽 얽를 빗헤 끓로 겄토 찐넬 약갼 끓로 쩜또 갛셩삐 잎뉘닫 멕쥬 닒담 뻘뤠 끄레 굵렴 빻로 엠많 떼익 펀얀학궤 할룽 눈눼 얌치 엠많 까윕 읊룔 돼얽 페졍 환뷸헤 라꼬오 구론 껴위 역김 씁뉘탁 언합눈 수윕 닭략교 좋숩님따 실는 끓로 뤼얻 틀롱 라꼬오 찜태됴 할룽 닌딸 듀곳 륄랏 라꼬오 슐련 총루 끼엿 쑥뺙 1쉬깐 찐넬 닌딸 닌딸 라꼬오 청톱 꽁싸 맏윈 꽂셰 쉭국 쟐묫 있골 섧요 똥홧 빵엑 휄쑤 구낭 쒸딱 냘랴 갖촉 맛 겆조 욕씰 좋앗움 터렵교 꼿익 까윕 긺랏 키량 샤쳔 햐씬눈 묫한는쥐 웍 앉낼 옰께오 총루 앉쿄 엊쪄 숩닉 꼿익 까윕 데퓌 땀베 까윕 높쥐 환뷸헤 헷아 예여 적긁 엔썽 라꼬오 냐빳 언는 두립 쥬뻔 잎뉘닫 뗄러 라꼬오 라꼬오 닌딸 듦엇 께욘 앉닒먼 실는 걋탸 딸위 퇴는 헷섬 캬눈 탐교 뮨웨 꼬쓴 뿐읾랴 좋얌오 라꼬오 록교 떼익 라꼬오 맛 쒸였 총루 터렵교 엠많 환뷸헤 눅킴 맛 좋야옥 빗헤 굣엠 펼한 엷씸휘 굵

In [19]:
label_sample_logic = labels.clone().detach()
for token_id in label_sample_logic:
    text = tokenizer.id_to_token(token_id)
    print(text,end=' ')

콴 뤽까 쩡맑 챨 뙈 있꾜 싸쟝뉨 위 친쩔 핫쉼 . 동 퀴뤼 탸 딱 땀 닥 얀 붙여 잇옆 섶 좋앝 숨 . 똘 롯 엽 윕 얀뉠랴 쨔 솝 릴 얀 듦림 . 깍걱 동 꿴찬 교 졍먈 만촉한 굣 . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <